#11. PySP Overview

This chapter describes PySP: (Pyomo Stochastic Programming), where parameters are allowed to be uncertain.

###11.1. Overview of Modeling Components and Processes
The sequence of activities is typically the following:

- Create a deterministic model and declare components

- Develop base-case data for the deterministic model

- Test, verify and validate the deterministic model

- Model the stochastic processes

- Develop a way to generate scenarios (in the form of a tree if there are more than two stages)

- Create the data files need to describe the stochastics

- Use PySP to solve stochastic problem

When viewed from the standpoint of file creation, the process is

- Create an abstract model for the deterministic problem in a file called <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span>

- Specify the stochastics in a file called <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span>

- Specify scenario data

###11.2. Birge and Louveaux’s Farmer Problem
Birge and Louveaux [BirgeLouveauxBook](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#BirgeLouveauxBook) make use of the example of a farmer who has 500 acres that can be planted in wheat, corn or sugar beets, at a per acre cost of 150, 230 and 260 (Euros, presumably), respectively. The farmer needs to have at least 200 tons of wheat and 240 tons of corn to use as feed, but if enough is not grown, those crops can be purchased for 238 and 210, respectively. Corn and wheat grown in excess of the feed requirements can be sold for 170 and 150, respectively. A price of 36 per ton is guaranteed for the first 6000 tons grown by any farmer, but beets in excess of that are sold for 10 per ton. The yield is 2.5, 3, and 20 tons per acre for wheat, corn and sugar beets, respectively.

#####11.2.1. ReferenceModel.py

So far, this is a deterministic problem because we are assuming that we know all the data. The Pyomo model for this problem shown here is in the file <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span> in the sub-directory <span style="color:darkblue; font-family:Courier">examples/pysp/farmer/models</span> that is distributed with Pyomo.



In [2]:
# Farmer: rent out version has a scalar root node var
# note: this will minimize
#
# Imports
#

from __future__ import division
from pyomo.environ import *

#
# Model
#

model = AbstractModel()

#
# Parameters
#

model.CROPS = Set()

model.TOTAL_ACREAGE = Param(within=PositiveReals)

model.PriceQuota = Param(model.CROPS, within=PositiveReals)

model.SubQuotaSellingPrice = Param(model.CROPS, within=PositiveReals)

def super_quota_selling_price_validate (model, value, i):
    return model.SubQuotaSellingPrice[i] >= model.SuperQuotaSellingPrice[i]

model.SuperQuotaSellingPrice = Param(model.CROPS, validate=super_quota_selling_price_validate)

model.CattleFeedRequirement = Param(model.CROPS, within=NonNegativeReals)

model.PurchasePrice = Param(model.CROPS, within=PositiveReals)

model.PlantingCostPerAcre = Param(model.CROPS, within=PositiveReals)

model.Yield = Param(model.CROPS, within=NonNegativeReals)

#
# Variables
#

model.DevotedAcreage = Var(model.CROPS, bounds=(0.0, model.TOTAL_ACREAGE))

model.QuantitySubQuotaSold = Var(model.CROPS, bounds=(0.0, None))
model.QuantitySuperQuotaSold = Var(model.CROPS, bounds=(0.0, None))

model.QuantityPurchased = Var(model.CROPS, bounds=(0.0, None))

model.FirstStageCost = Var()
model.SecondStageCost = Var()

#
# Constraints
#

def ConstrainTotalAcreage_rule(model):
    return summation(model.DevotedAcreage) <= model.TOTAL_ACREAGE

model.ConstrainTotalAcreage = Constraint(rule=ConstrainTotalAcreage_rule)

def EnforceCattleFeedRequirement_rule(model, i):
    return model.CattleFeedRequirement[i] <= (model.Yield[i] * model.DevotedAcreage[i]) + model.QuantityPurchased[i] - model.QuantitySubQuotaSold[i] - model.QuantitySuperQuotaSold[i]

model.EnforceCattleFeedRequirement = Constraint(model.CROPS, rule=EnforceCattleFeedRequirement_rule)

def LimitAmountSold_rule(model, i):
    return model.QuantitySubQuotaSold[i] + model.QuantitySuperQuotaSold[i] - (model.Yield[i] * model.DevotedAcreage[i]) <= 0.0

model.LimitAmountSold = Constraint(model.CROPS, rule=LimitAmountSold_rule)

def EnforceQuotas_rule(model, i):
    return (0.0, model.QuantitySubQuotaSold[i], model.PriceQuota[i])

model.EnforceQuotas = Constraint(model.CROPS, rule=EnforceQuotas_rule)

#
# Stage-specific cost computations
#

def ComputeFirstStageCost_rule(model):
    return model.FirstStageCost - summation(model.PlantingCostPerAcre, model.DevotedAcreage) == 0.0

model.ComputeFirstStageCost = Constraint(rule=ComputeFirstStageCost_rule)

def ComputeSecondStageCost_rule(model):
    expr = summation(model.PurchasePrice, model.QuantityPurchased)
    expr -= summation(model.SubQuotaSellingPrice, model.QuantitySubQuotaSold)
    expr -= summation(model.SuperQuotaSellingPrice, model.QuantitySuperQuotaSold)
    return (model.SecondStageCost - expr) == 0.0

model.ComputeSecondStageCost = Constraint(rule=ComputeSecondStageCost_rule)

#
# Objective
#

def Total_Cost_Objective_rule(model):
    return model.FirstStageCost + model.SecondStageCost

model.Total_Cost_Objective = Objective(sense=minimize, rule=Total_Cost_Objective_rule)

#####11.2.2. Example Data

The data introduced here are in the file ReferenceModel.dat in the sub-directory examples/pysp/farmer/scenariodata that is distributed with Pyomo. These data are given for illustration. The file ReferenceModel.dat is not required by PySP.

In [3]:
set CROPS := WHEAT CORN SUGAR_BEETS ;

param TOTAL_ACREAGE := 500 ;

# no quotas on wheat or corn
param PriceQuota :=
            WHEAT 100000 CORN 100000 SUGAR_BEETS 6000 ;

param SubQuotaSellingPrice :=
            WHEAT 170 CORN 150 SUGAR_BEETS 36 ;

param SuperQuotaSellingPrice :=
            WHEAT 0 CORN 0 SUGAR_BEETS 10 ;

param CattleFeedRequirement :=
            WHEAT 200 CORN 240 SUGAR_BEETS 0 ;

# can't purchase beets (no need, as cattle don't eat them)
param PurchasePrice :=
            WHEAT 238 CORN 210 SUGAR_BEETS 100000 ;

param PlantingCostPerAcre :=
            WHEAT 150 CORN 230 SUGAR_BEETS 260 ;

param Yield := WHEAT 3.0 CORN 3.6 SUGAR_BEETS 24 ;

SyntaxError: invalid syntax (<ipython-input-3-16a4ff118bd2>, line 1)

Any of these data could be modeled as uncertain, but we will consider only the possibility that the yield per acre could be higher or lower than expected. Assume that there is a probability of 1/3 that the yields will be the average values that were given (i.e., wheat 2.5; corn 3; and beets 20). Assume that there is a 1/3 probability that they will be lower (2, 2.4, 16) and 1/3 probability they will be higher (3, 3.6, 24). We refer to each full set of data as a <span style="color:darkblue">*scenario*</span> and collectively we call them a <span style="color:darkblue">*scenario tree*</span>. In this case the scenario tree is very simple: there is a root node and three leaf nodes: one corresponding to each scenario. The acreage-to-plant decisions are root node decisions because they must be made without knowing what the yield will be. The other variables are so-called <span style="color:darkblue">*second stage*</span> decisions, because they will depend on which scenario is realized.

#####11.2.3. ScenarioStructure.dat

PySP requires that users describe the scenario tree using specific constructs in a file named <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat;</span> for the farmer problem, this file can be found in the pyomo sub-directory <span style="color:darkblue; font-family:Courier">examples/pysp/farmer/scenariodata</span> that is distributed with Pyomo.



In [4]:
# IMPORTANT - THE STAGES ARE ASSUMED TO BE IN TIME-ORDER.

set Stages := FirstStage SecondStage ;

set Nodes := RootNode
             BelowAverageNode
             AverageNode
             AboveAverageNode ;

param NodeStage := RootNode         FirstStage
                   BelowAverageNode SecondStage
                   AverageNode      SecondStage
                   AboveAverageNode SecondStage ;

set Children[RootNode] := BelowAverageNode
                          AverageNode
                          AboveAverageNode ;

param ConditionalProbability := RootNode          1.0
                                BelowAverageNode  0.33333333
                                AverageNode       0.33333334
                                AboveAverageNode  0.33333333 ;

set Scenarios := BelowAverageScenario
                 AverageScenario
                 AboveAverageScenario ;

param ScenarioLeafNode :=
                    BelowAverageScenario BelowAverageNode
                    AverageScenario      AverageNode
                    AboveAverageScenario AboveAverageNode ;

set StageVariables[FirstStage] :=  DevotedAcreage[*] ;
set StageVariables[SecondStage] := QuantitySubQuotaSold[*]
                                   QuantitySuperQuotaSold[*]
                                   QuantityPurchased[*] ;

param StageCostVariable := FirstStage  FirstStageCost
                           SecondStage SecondStageCost ;


SyntaxError: invalid syntax (<ipython-input-4-4de393e08abc>, line 3)

This data file is verbose and somewhat redundant, but in most applications it is generated by software rather than by a person, so this is not an issue. Generally, the left-most part of each expression (e.g. “set Stages :=”) is required and uses reserved words (e.g., <span style="color:darkblue; font-family:Courier">Stages</span>) and the other names are supplied by the user (e.g., “FirstStage” could be any name). Every assignment is terminated with a semi-colon. We will now consider the assignments in this file one at a time.

The first assignments provides names for the stages and the words "set Stages" are required, as are the := symbols. Any names can be used. In this example, we used "FirstStage" and "SecondStage" but we could have used "EtapPrimero" and "ZweiteEtage" if we had wanted to. Whatever names are given here will continue to be used to refer to the stages in the rest of the file. The order of the names is important. A simple way to think of it is that generally, the names must be in time order (technically, they need to be in order of information discovery, but that is usually time-order). Stages refers to decision stages, which may, or may not, correspond directly with time stages. In the farmer example, decisions about how much to plant are made in the first stage and "decisions" (which are pretty obvious, but which are decision variables nonetheless) about how much to sell at each price and how much needs to be bought are second stage decisions because they are made after the yield is known.



In [5]:
set Stages := FirstStage SecondStage ;

SyntaxError: invalid syntax (<ipython-input-5-36b09a7decf7>, line 1)

Node names are constructed next. The words "set Nodes" are required, but any names may be assigned to the nodes. In two stage stochastic problems there is a root node, which we chose to name "RootNode" and then there is a node for each scenario.

In [6]:
set Nodes := RootNode
             BelowAverageNode
             AverageNode
             AboveAverageNode ;

SyntaxError: invalid syntax (<ipython-input-6-9ce9b5a3bd03>, line 1)

Nodes are associated with time stages with an assignment beginning with the required words "param Nodestage." The assignments must make use of previously defined node and stage names. Every node must be assigned a stage.

In [7]:
param NodeStage := RootNode         FirstStage
                   BelowAverageNode SecondStage
                   AverageNode      SecondStage
                   AboveAverageNode SecondStage ;

SyntaxError: invalid syntax (<ipython-input-7-fda53ee75bc6>, line 1)

The structure of the scenario tree is defined using assignment of children to each node that has them. Since this is a two stage problem, only the root node has children. The words "param Children" are required for every node that has children and the name of the node is in square brackets before the colon-equals assignment symbols. A list of children is assigned.

In [10]:
set Children[RootNode] := BelowAverageNode
                          AverageNode
                          AboveAverageNode ;

SyntaxError: invalid syntax (<ipython-input-10-ca4afaaf92cf>, line 1)

The probability for each node, conditional on observing the parent node is given in an assignment that begins with the required words "param ConditionalProbability." The root node always has a conditional probability of 1, but it must always be given anyway. In this example, the second stage nodes are equally likely.

In [11]:
param ConditionalProbability := RootNode          1.0
                                BelowAverageNode  0.33333333
                                AverageNode       0.33333334
                                AboveAverageNode  0.33333333 ;

SyntaxError: invalid syntax (<ipython-input-11-b18b2ab7cd04>, line 1)

Scenario names are given in an assignment that begins with the required words "set Scenarios" and provides a list of the names of the scenarios. Any names may be given. In many applications they are given unimaginative names generated by software such as "Scen1" and the like. In this example, there are three scenarios and the names reflect the relative values of the yields.

In [12]:
set Scenarios := BelowAverageScenario
                 AverageScenario
                 AboveAverageScenario ;

SyntaxError: invalid syntax (<ipython-input-12-a764fad394fc>, line 1)

Leaf nodes, which are nodes with no children, are associated with scenarios. This assignment must be one-to-one and it is initiated with the words "param ScenarioLeafNode" followed by the colon-equals assignment characters.

In [14]:
param ScenarioLeafNode :=
                    BelowAverageScenario BelowAverageNode
                    AverageScenario      AverageNode
                    AboveAverageScenario AboveAverageNode ;

SyntaxError: invalid syntax (<ipython-input-14-06029a5de46d>, line 1)

Variables are associated with stages using an assignment that begins with the required words "set StageVariables" and the name of a stage in square brackets followed by the colon-equals assignment characters. Variable names that have been defined in the file ReferenceModel.py can be assigned to stages. Any variables that are not assigned are assumed to be in the last stage. Variable indexes can be given explicitly and/or wildcards can be used. Note that the variable names appear without the prefix "model." In the farmer example, DevotedAcreage is the only first stage variable.

In [15]:
set StageVariables[FirstStage] :=  DevotedAcreage[*] ;
set StageVariables[SecondStage] := QuantitySubQuotaSold[*]
                                   QuantitySuperQuotaSold[*]
                                   QuantityPurchased[*] ;

SyntaxError: invalid syntax (<ipython-input-15-2121d4b3b75a>, line 1)


>#####Note
>Variable names appear without the prefix "model."

>#####Note
>Wildcards can be used, but fully general Python slicing is not supported.

For reporting purposes, it is useful to define auxiliary variables in <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span> that will be assigned the cost associated with each stage. This variables do not impact algorithms, but the values are output by some software during execution as well as upon completion. The names of the variables are assigned to stages using the "param StageCostVariable" assignment. The stages are previously defined in <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> and the variables are previously defined in <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span>.



In [18]:
param StageCostVariable := FirstStage  FirstStageCost
                           SecondStage SecondStageCost ;

SyntaxError: invalid syntax (<ipython-input-18-fd61f26ba646>, line 1)

#####11.2.4. Scenario data specification

So far, we have given a model in the file named <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span>, a set of deterministic data in the file named <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span>, and a description of the stochastics in the file named <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span>. All that remains is to give the data for each scenario. There are two ways to do that in PySP: <span style="color:darkblue">*scenario-based*</span> and <span style="color:darkblue">*node-based*</span>. The default is scenario-based so we will describe that first.

For scenario-based data, the full data for each scenario is given in a <span style="color:darkblue; font-family:Courier">.dat</span> file with the root name that is the name of the scenario. So, for example, the file named <span style="color:darkblue; font-family:Courier">AverageScenario.dat</span> must contain all the data for the model for the scenario named "AvererageScenario." It turns out that this file can be created by simply copying the file <span style="color:darkblue; font-family:Courier">ReferenceModel.dat</span> as shown above because it contains a full set of data for the "AverageScenario" scenario. The files <span style="color:darkblue; font-family:Courier">BelowAverageScenario.dat</span> and <span style="color:darkblue; font-family:Courier">AboveAverageScenario.dat</span> will differ from this file and from each other only in their last line, where the yield is specified. These three files are distributed with Pyomo and are in the pyomo sub-directory <span style="color:darkblue; font-family:Courier">examples/pysp/farmer/scenariodata</span> along with <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> and <span style="color:darkblue; font-family:Courier">ReferenceModel.dat</span>.

Scenario-based data wastes resources by specifying the same thing over and over again. In many cases, that does not matter and it is convenient to have full scenario data files available (for one thing, the scenarios can easily be run independently using the <span style="color:darkblue; font-family:Courier">pyomo</span> command). However, in many other settings, it is better to use a node-based specification where the data that is unique to each node is specified in a <span style="color:darkblue; font-family:Courier">.dat</span> file with a root name that matches the node name. In the farmer example, the file <span style="color:darkblue; font-family:Courier">RootNode.dat</span> will be the same as <span style="color:darkblue; font-family:Courier">ReferenceModel.dat</span> except that it will lack the last line that specifies the yield. The files <span style="color:darkblue; font-family:Courier">BelowAverageNode.dat</span>, <span style="color:darkblue; font-family:Courier">AverageNode.dat</span>, and <span style="color:darkblue; font-family:Courier">AboveAverageNode.dat</span> will contain only one line each to specify the yield. If node-based data is to be used, then the <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> file must contain the following line:

In [19]:
param ScenarioBasedData := False ;

SyntaxError: invalid syntax (<ipython-input-19-4661a4ebfaa0>, line 1)

An entire set of files for node-based data for the farmer problem are distributed with Pyomo in the sub-directory <span style="color:darkblue; font-family:Courier">examples/pysp/farmer/nodedata</span>

###11.3. Finding Solutions for Stochastic Models
PySP provides a variety of tools for finding solutions to stochastic programs.

#####11.3.1. runef

The <span style="color:darkblue; font-family:Courier">runef</span> command puts together the so-called <span style="color:darkblue">*extensive form*</span> version of the model. It creates a large model that has constraints to ensure that variables at a node have the same value. For example, in the farmer problem, all of the <span style="color:darkblue; font-family:Courier">DevotedAcres</span> variables must have the same value regardless of which scenario is ultimately realized. The objective can be the expected value of the objective function, or the CVaR, or a weighted combination of the two. Expected value is the default. A full set of options for <span style="color:darkblue; font-family:Courier">runef</span> can be obtained using the command:



In [1]:
!runef --help

Usage: runef [options]

Options:
  -h, --help            show this help message and exit

  Input Options:
    -i INSTANCE_DIRECTORY, --instance-directory=INSTANCE_DIRECTORY
                        The directory in which all instance (reference and
                        scenario) definitions are stored. This option is
                        required if no callback is found in the model file.
    -m MODEL_DIRECTORY, --model-directory=MODEL_DIRECTORY
                        The directory in which all model (reference and
                        scenario) definitions are stored. Default is ".".
    -o OBJECTIVE_SENSE, --objective-sense-stage-based=OBJECTIVE_SENSE
                        The objective sense to use for the auto-generated
                        scenario instance objective, which is equal to the sum
                        of the scenario-tree stage costs. Default is None,
                        indicating an Objective has been declared on the
                        ref

The pyomo distribution contains the files need to run the farmer example in the sub-directories to the sub-directory <span style="color:darkblue; font-family:Courier">examples/pysp/farmer</span> so if this is the current directory and if CPLEX is installed, the following command will cause formation of the EF and its solution using CPLEX.

In [2]:
!runef -m models -i nodedata --solver=cplex --solve

***ERROR: Failed to locate reference model file with specification string: models
RUN-TIME ERROR: Model input does not exist: /Users/rhart/PyomoGettingStarted/models/ReferenceModel.py


The option <span style="color:darkblue; font-family:Courier">-m models</span> has one dash and is short-hand for the option <span style="color:darkblue; font-family:Courier">--model-directory=models</span> and note that the full option uses two dashes. The <span style="color:darkblue; font-family:Courier">-i</span> is equivalent to <span style="color:darkblue; font-family:Courier">--instance-directory=</span> in the same fashion. The default solver is CPLEX, so the solver option is not really needed. With the <span style="color:darkblue; font-family:Courier">--solve</span> option, runef would simply write an .lp data file that could be passed to a solver.

#####11.3.2. runph

The runph command executes an implementation of Progressive Hedging (PH) that is intended to support scripting and extension.

The pyomo distribution contains the files need to run the farmer example in the sub-directories to the sub-directory examples/pysp/farmer so if this is the current directory and if CPLEX is installed, the following command will cause PH to execute using the default sub-problem solver, which is CPLEX.



In [3]:
!runph -m models -i nodedata

***ERROR: Failed to locate reference model file with specification string: models


To obtain further information regarding the source of the exception, use the --traceback option


The option <span style="color:darkblue; font-family:Courier">-m models</span> has one dash and is short-hand for the option <span style="color:darkblue; font-family:Courier">--model-directory=models</span> and note that the full option uses two dashes. The -i is equivalent to <span style="color:darkblue; font-family:Courier">--instance-directory=</span> in the same fashion.

After about 33 iterations, the algorithm will achieve the default level of convergence and terminate. A lot of output is generated and among the output is the following solution information:

In [23]:
Variable=DevotedAcreage
        Index: [CORN]            (Scenarios:  BelowAverageScenario   AverageScenario   AboveAverageScenario   )
                Values:       79.9844      80.0000      79.9768     Max-Min=      0.0232     Avg=     79.9871
        Index: [SUGAR_BEETS]             (Scenarios:  BelowAverageScenario   AverageScenario   AboveAverageScenario   )
                Values:      249.9848     249.9770     250.0000     Max-Min=      0.0230     Avg=    249.9873
        Index: [WHEAT]           (Scenarios:  BelowAverageScenario   AverageScenario   AboveAverageScenario   )
                Values:      170.0308     170.0230     170.0232     Max-Min=      0.0078     Avg=    170.0256
Cost Variable=FirstStageCost
        Tree Node=RootNode               (Scenarios:  BelowAverageScenario   AverageScenario   AboveAverageScenario   )
        Values:   108897.0836  108897.4725  108898.1476     Max-Min=      1.0640     Avg= 108897.5679


IndentationError: unexpected indent (<ipython-input-23-6edb9f091fd5>, line 2)

For problems with no, or few, integer variables, the default level of convergence leaves root-node variables almost converged. Since the acreage to be planted cannot depend on the scenario that will be realized in the future, the average, which is labeled "Avg" in this output, would be used. A farmer would probably interpret acreages of 79.9871, 249.9873, and 170.0256 to be 80, 250, and 170. In real-world applications, PH is embedded in scripts that produce output in a format desired by a decision maker.

But in real-world applications, the default settings for PH seldom work well enough. In addition to post-processing the output, a number of parameters need to be adjusted and sometimes scripting to extend or augment the algorithm is needed to improve convergence rates. A full set of options can be obtained with the command:

In [4]:
!runph --help

Usage: runph [options]

Options:
  -h, --help            show this help message and exit

  Input Options:
    -m MODEL_DIRECTORY, --model-directory=MODEL_DIRECTORY
                        The directory in which all model (reference and
                        scenario) definitions are stored. Default is ".".
    -i INSTANCE_DIRECTORY, --instance-directory=INSTANCE_DIRECTORY
                        The directory in which all instance (reference and
                        scenario) definitions are stored. This option is
                        required if no callback is found in the model file.
    -o OBJECTIVE_SENSE, --objective-sense-stage-based=OBJECTIVE_SENSE
                        The objective sense to use for the auto-generated
                        scenario instance objective, which is equal to the sum
                        of the scenario-tree stage costs. Default is None,
                        indicating an Objective has been declared on the
                        ref

Note that there are two dashes before <span style="color:darkblue; font-family:Courier">help</span>.

By default, PH uses quadratic objective functions after iteration zero; in some settings it may be desirable to linearize the quadratic terms. This is required to use a solver such as glpk for MIPs because it does not support quadratic MIPs. The directive <span style="color:darkblue; font-family:Courier">--linearize-nonbinary-penalty-terms=n</span> causes linearization of the penalty terms using n pieces. For example, to use glpk on the farmer, assuming glpk is installed and the command is given when the current directory is the <span style="color:darkblue; font-family:Courier">examples/pysp/farmer</span>, the following command will use default settings for most parameters and four pieces to approximate quadratic terms in sub-problems:

In [5]:
!runph -i nodedata -m models --solver=glpk --linearize-nonbinary-penalty-terms=4

***ERROR: Failed to locate reference model file with specification string: models


To obtain further information regarding the source of the exception, use the --traceback option


Use of the <span style="color:darkblue; font-family:Courier">linearize-nonbinary-penalty-terms</span> option requires that all variables not in the final stage have bounds.

####11.3.3 Final Solution

At each iteration, PH computes an average for each variable over the nodes of the scenario tree. We refer to this as X-bar. For many problems, particularly those with integer restrictions, X-bar might not be feasible for every scenario unless PH happens to be fully converged (in the primal variables). Consequently, the software computes a solution system X-hat that is more likely to be feasible for every scenario and will be equivalent to X-bar under full convergence. This solution is reported upon completion of PH and its expected value is report if it is feasible for all scenarios.

Methods for computing X-hat are controlled by the <span style="color:darkblue; font-family:Courier">l--xhat-method</span> command-line option. For example

In [6]:
--xhat-method=closest-scenario

SyntaxError: can't assign to operator (<ipython-input-6-55e9f709cf38>, line 1)

causes X-hat to be set to the scenario that is closest to X-bar (in a z-score sense). Other options, such as <span style="color:darkblue; font-family:Courier">voting</span> and <span style="color:darkblue; font-family:Courier">rounding</span>, assign values of X-bar to X-hat except for binary and general integer variables, where the values are set by probability weighted voting by scenarios and rounding from X-bar, respectively.

####11.3.4. Solution Output Control

To get the full solution, including leaf node solution values, use the <span style="color:darkblue; font-family:Courier">runph --output-scenario-tree-solution</span> option.

In both runph and runef the solution can be written in csv format using the <span style="color:darkblue; font-family:Courier">--solution-writer=pyomo.pysp.plugins.csvsolutionwriter</span> option.

###11.4. Summary of PySP File Names
PySP scripts such as <span style="color:darkblue; font-family:Courier">runef</span> and <span style="color:darkblue; font-family:Courier">runph</span> require files that specify the model and data using files with specific names. All files can be in the current directory, but typically, the file <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span> is in a directory that is specified using <span style="color:darkblue; font-family:Courier">--model-directory=</span> option (the short version of this option is <span style="color:darkblue; font-family:Courier">-i +) and the data files are in a directory specified in the +--instance-directory=</span> option (the short version of this option is +-m +).

>#####Note
>A file name other than <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span> can be used if the file name is given in addition to the directory name as an argument to the <span style="color:darkblue; font-family:Courier">--instance-directory</span> option. For example, on a Windows machine <span style="color:darkblue; font-family:Courier">--instance-directory=models\MyModel.py</span> would specify the file <span style="color:darkblue; font-family:Courier">MyModel.py</span> in the local directory <span style="color:darkblue; font-family:Courier">models</span>.

- <span style="color:darkblue; font-family:Courier">ReferenceModel.py</span>: A full Pyomo model for a singe scenario. There should be no scenario indexes in this model because they are implicit.

- <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span>: Specifies the nature of the stochastics. It also specifies whether the rest of the data is node-based or scenario-based. It is scenario-based unless <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> contains the line



In [26]:
param ScenarioBasedData := False ;

SyntaxError: invalid syntax (<ipython-input-26-4661a4ebfaa0>, line 1)

If scenario-based, then there is a data file for each scenario that specifies a full set of data for the scenario. The name of the file is the name of the scenario with <span style="color:darkblue; font-family:Courier">.dat</span> appended. The names of the scenarios are given in the <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> file.

If node-based, then there is a file with data for each node that specifies only that data that is unique for the node. The name of the file is the name of the node with <span style="color:darkblue; font-family:Courier">.dat</span> appended. The names of the nodes are given in the <span style="color:darkblue; font-family:Courier">ScenarioStructure.dat</span> file.

###11.5. Solving Sub-problems in Parallel and/or Remotely
The Python package called Pyro provides capalities that are used to enable PH to make use of multiple solver processes for sub-problems and allows both <span style="color:darkblue; font-family:Courier">runef</span> and <span style="color:darkblue; font-family:Courier">runph</span> to make use remote solvers. We will focus on PH in our discussion here.

There are two solver management systems available for <span style="color:darkblue; font-family:Courier">runph</span>, one is based on a <span style="color:darkblue; font-family:Courier">pyro_mip_server</span> and the other is based on a <span style="color:darkblue; font-family:Courier">phsolverserver</span>. Regardless of which is used, a name server and a dispatch server must be running and accessible to the <span style="color:darkblue; font-family:Courier">runph</span> process. The name server is launched using the command <span style="color:darkblue; font-family:Courier">pyomo_ns</span> and then the dispatch server is launched with <span style="color:darkblue; font-family:Courier">dispatch_srvr</span>. Note that both commands contain an underscore. Both programs keep running until terminated by an external signal, so it is common to pipe their output to a file.

Solvers are controlled by solver servers. The pyro mip solver server is launched with the command <span style="color:darkblue; font-family:Courier">pyro_mip_server</span>. This command may be repeated to launch as many solvers as are desired. The <span style="color:darkblue; font-family:Courier">runph</span> then needs a <span style="color:darkblue; font-family:Courier">--solver-manager=pyro</span> option to signal that <span style="color:darkblue; font-family:Courier">runph</span> should not launch its own solver, but should send subproblems to be dispatched to parallel solvers. To summarize the commands:

- Once: <span style="color:darkblue; font-family:Courier">pyomo_ns</span>

- Once: <span style="color:darkblue; font-family:Courier">dispatch_srvr</span>

- Multiple times:<span style="color:darkblue; font-family:Courier"> pyro_mip_server</span>

- Once: <span style="color:darkblue; font-family:Courier">runph … --solver-manager=pyro …</span>

>#####Note
>The <span style="color:darkblue; font-family:Courier">runph</span> option <span style="color:darkblue; font-family:Courier">--shutdown-pryo</span> will cause a shutdown signal to be sent to <span style="color:darkblue; font-family:Courier">pyomo_ns, dispatch_srvr</span> and all <span style="color:darkblue; font-family:Courier">pyro_mip_server</span> programs upon termination of <span style="color:darkblue; font-family:Courier">runph</span>.

Instead of using <span style="color:darkblue; font-family:Courier">pyro_mip_server</span>, one can use <span style="color:darkblue; font-family:Courier">phsolverserver</span> in its place. You can get a list of arguments using <span style="color:darkblue; font-family:Courier">pyrosolverserver --help</span>, which does not launch a solver server (it just displays help and terminates). If you use the phsolversover, then use <span style="color:darkblue; font-family:Courier">--solver-manager=phpyro</span> as an argument to runph rather than <span style="color:darkblue; font-family:Courier">--solver-manager=pyro</span>.

>#####Warning
>Unlike the normal <span style="color:darkblue; font-family:Courier">pyro_mip_server</span>, there must be one <span style="color:darkblue; font-family:Courier">phsolverserver</span> for each sub-problem. One can use fewer phsolverservers than there are scenarios by adding the command-line option “--phpyro-required-workers=X”. This will partition the jobs among the available workers,
